# V18: Sensitivity sweep (detrend/variant robustness)

Runs the detection metric across a grid of preprocessing variants (downsampling, outlier removal, detrending) to assess robustness.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V18')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': list(r.flags),
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "best_variant_id": "ds1|ol_none|dt_none",
    "depth_spread_iqr_over_median": 0.47455256303888355,
    "metric_variance": 0.4817938764839944,
    "n_variants_failed": 0,
    "n_variants_ok": 12,
    "n_variants_total": 12,
    "score_spread_iqr_over_median": 0.4817938764839944,
    "worst_variant_id": "ds5|ol_sigma_clip_4|dt_running_median_0.5d"
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_sensitivity_sweep

step_id = '20_v18_sensitivity_sweep'
fname = 'V18_sensitivity_sweep.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_sensitivity_sweep(r, ax=ax)
    ax.set_title('V18: Sensitivity sweep')
    fig.tight_layout()
    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/20_v18_sensitivity_sweep/V18_sensitivity_sweep.png" width="820" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/20_v18_sensitivity_sweep/V18_sensitivity_sweep.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/20_v18_sensitivity_sweep/V18_sensitivity_sweep.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** score and depth vary substantially across variants (IQR/median ≈ 0.48 for score; ≈ 0.47 for depth).
- **Why this is useful:** a planet-like signal should generally persist (within reason) under small analysis changes; strong dependence can indicate marginal SNR, systematics sensitivity, or sector-specific artifacts.
- **Interpretation:** this level of spread is a warning that conclusions will depend on analysis choices; proceed by identifying which variants/sector sets are driving instability.
- **Next step:** V19 alias diagnostics to see whether harmonics/aliases compete strongly with the base period.

</details>
